The following notebook adapts some code from the TensorFlow tutorial: https://www.tensorflow.org/tutorials/keras/classification

In [1]:
import keras
from keras import Sequential
from keras.layers  import Dense, Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Helper libraries
import numpy as np  # Keras needs an older version on numpy
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
from RProp import RProp
from Wame import Wame

## Import Landsat

In [3]:
def extract_data_lables(file):
    data, lables = [], []
    with open(file, "r") as f:
        for row in f:
            row = row.split()
            row = [int(i) for i in row]
            lables.append(row.pop(-1))
            data.append(row)
        return np.array(data), np.array(lables)

In [4]:
train_data, train_lables = extract_data_lables("data\landsat\sat.trn")

In [5]:
test_data, test_lables  = extract_data_lables("data\landsat\sat.tst")

In [6]:
print(len(train_data))
print(len(test_data))

4435
2000


### Clean Lables

In [7]:
train_lables -= 1
test_lables -= 1

In [8]:
class_names = {
    0:'red soil'
    ,1:'cotton crop'
    ,2:'grey soil'
    ,3:'damp grey soil'
    ,4:'soil with vegetation stubble'
    ,5:'mixture class (all types present)'
    ,6:'very damp grey soil'
}

## Preprocess Images

In [9]:
# scale pixels to 0-1
train_data = train_data / 255.0
test_data = test_data / 255.0

TODO: Find a way to optimise these layers (will be needed for report)

## One-hot encode lables
Class lables need to be in an nxc array format

In [10]:
from sklearn.preprocessing import OneHotEncoder

In [11]:
ohe = OneHotEncoder(sparse=False)
train_y = ohe.fit_transform(train_lables.reshape(-1, 1))
test_y = ohe.transform(test_lables.reshape(-1, 1))

In [12]:
train_data.shape

(4435, 36)

In [13]:
train_y.shape

(4435, 6)

## Functions for creating and testing models

In [14]:
def train_test_model(model, optimiser='adam'):
    """Trains a model using 25% validation, then tests the model with the best validation performance"""
    
    model.compile(optimizer=optimiser
              ,loss='categorical_crossentropy'
              ,metrics=['accuracy']
             )
    
    history =  model.fit(
        train_data
        , train_y
        , epochs=50
        , validation_split=0.25
        , verbose=0
        ,callbacks = [
            EarlyStopping(verbose=False, patience=20, monitor='val_loss'),
            ModelCheckpoint('TestModel-progress', monitor='val_loss', verbose=False, save_best_only=True)],
    )
    
    test_loss, test_acc = model.evaluate(test_data,  test_y, verbose = 0)
    return (test_loss, test_acc)   

In [15]:
def create_seq_model(n_hidden=1, n_nodes=36):
    model = Sequential()
    # First hidden layer
    model.add(Dense(n_nodes, input_dim=36))  # input_dim easier to use than input_shape
    model.add(Activation('relu'))
    # Additional hidden layers
    for i in range(n_hidden-1):
        model.add(Dense(n_nodes))
        model.add(Activation('relu'))
    # output layer
    model.add(Dense(6))  
    model.add(Activation('sigmoid'))
    return model

In [16]:
def train_test_grid(n_hidden_lst, n_nodes_lst, optimiser):
    test_losses = np.zeros((len(n_hidden_lst), len(n_nodes_lst)))
    test_accs = np.zeros((len(n_hidden_lst), len(n_nodes_lst)))

    for i, n_hidden in enumerate(n_hidden_lst):
        for j, n_nodes in enumerate(n_nodes_lst):
            model = create_seq_model(n_hidden, n_nodes)
            test_result = train_test_model(model, optimiser)
            test_losses[i, j] = test_result[0]
            test_accs[i, j] = test_result[1]
    
    return test_accs, test_losses

#### Initial model check

In [17]:
from keras.callbacks import LambdaCallback

In [18]:
model = create_seq_model(1, 36)

print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[0].get_weights()))

model.compile(optimizer=Wame(clipnorm=1.0)
              ,loss='categorical_crossentropy'
              ,metrics=['accuracy']
             )
model.fit(
        train_data
        , train_y
        , epochs=50
        , validation_split=0.25
        , verbose=0
        , callbacks = [print_weights]
    )
    
test_loss, test_acc = model.evaluate(test_data,  test_y, verbose = 0)

print(test_acc)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
[array([[ 0.09354419,  0.11822214,  0.1814964 , ..., -0.27013838,
         0.07642087,  0.1466817 ],
       [ 0.1290336 ,  0.12220987, -0.09291027, ..., -0.13424657,
        -0.07629868,  0.04724776],
       [-0.19451928, -0.2761355 ,  0.09240671, ..., -0.08420589,
         0.09345121,  0.21572296],
       ...,
       [ 0.20361802,  0.14183709,  0.0206915 , ..., -0.17240782,
        -0.02026012, -0.30092713],
       [ 0.03850988, -0.0416666 ,  0.10446534, ...,  0.1201624 ,
        -0.05324903,  0.23076618],
       [-0.27986202,  0.21854928,  0.0947246 , ...,  0.02332178,
         0.19545431, -0.10415749]], dtype=float32), array([ 0.        , -0.08093237, -0.16580275,  0.03097752, -0.12857912,
       -0.12162715,  0.        , -0.02147406,  0.        , -0.5109967 ,
        0.        ,  0.        ,  0.        ,  0.        , -0.0618774 ,
       -0.1467121 , -0.10553797, -

[array([[ 0.09354419,  0.11822214,  0.1814964 , ..., -0.27013838,
         0.07642087,  0.1466817 ],
       [ 0.1290336 ,  0.12220987, -0.09291027, ..., -0.13424657,
        -0.07629868,  0.04724776],
       [-0.19451928, -0.2761355 ,  0.09240671, ..., -0.08420589,
         0.09345121,  0.21572296],
       ...,
       [ 0.20361802,  0.14183709,  0.0206915 , ..., -0.17240782,
        -0.02026012, -0.30092713],
       [ 0.03850988, -0.0416666 ,  0.10446534, ...,  0.1201624 ,
        -0.05324903,  0.23076618],
       [-0.27986202,  0.21854928,  0.0947246 , ...,  0.02332178,
         0.19545431, -0.10415749]], dtype=float32), array([ 0.0000000e+00, -8.0932371e-02, -1.6580275e-01, -4.2339844e+01,
       -1.2857912e-01, -1.2162715e-01,  0.0000000e+00, -2.1533553e-02,
        0.0000000e+00, -5.1099670e-01,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00, -6.1877403e-02, -1.4671209e-01,
       -1.0553797e-01, -3.0852156e-03, -1.2397405e-01, -5.4012752e-01,
        0.00000

[array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)]
[array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan

       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)]
[array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)]
[array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, n

In [19]:
model.predict(train_data)

array([[nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       ...,
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan]], dtype=float32)

#### Test various 'adam' configurations

In [ ]:
n_hidden_lst = [1, 2, 3]
n_nodes_lst = [10, 20, 30, 40]
optimiser = 'adam'

In [ ]:
# test_accs_adam, test_losses_adam = train_test_grid(n_hidden_lst, n_nodes_lst, optimiser)
# print(test_accs_adam)

##### Test various 'wame' configurations

In [ ]:
test_accs_wame, test_losses_wame = train_test_grid(n_hidden_lst, n_nodes_lst, Wame())
print(test_accs_wame)

In [ ]:
# train_test_model(model)

In [ ]:
# model = create_seq_model(n_hidden=2)
# train_test_model(model)

### Train

In [ ]:
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Val'], loc='upper left')
# plt.show()